In [1]:
import pandas as pd
import numpy as np
import altair as alt
import os
# import chromatose as ct

import geopandas as gpd
from geopy.distance import geodesic
from itertools import product
import json

from IPython.display import SVG
import scipy.special

/Users/rf50/opt/anaconda3/lib/python3.9/site-packages/pandas/core/arrays/masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


### Read in data and re-define dictionaries
Note that L (workers), H (housing units), N (households), W (wages) are all in units of 1,000.

In [2]:
df_model_ij = pd.read_csv('processed_data/DF_MODEL_IJ.csv')
df_model_i = pd.read_csv('processed_data/DF_MODEL_I.csv')
df_model_j = pd.read_csv('processed_data/DF_MODEL_J.csv')

counties = [
    'Los Angeles', 'Orange', 'Riverside', 
    'San Bernardino', 'San Diego', 'Ventura']

d_index_county = {i: county for i, county in enumerate(counties)}
d_county_index = {v:k for k, v in d_index_county.items()}

d_county_fips = {
    'Los Angeles':'037',
    'Orange':'059',
    'Riverside':'065',
    'San Bernardino':'071',
    'San Diego':'073',
    'Ventura':'111',
}
d_fips_county = {v: k for k, v in d_county_fips.items()}

### Plan

We will build $d_{ij}$, $F_i$, $Z_j$ from wages.   
We will solve for baseline equilibrium parameters.   
Then increase $F_{LA}$, and re-solve for equilibrium.   
Compare population, wages, housing, and commuting.

At spatial equilibrium: 
- labor markets clear
- housing markets clear
- commuting flows reflect choices
  households are indifferent across residence-workplace pairs


### Construct $d_{ij}$

In [3]:
# First read in centroids from cleaned data
df_centroid = pd.read_csv('processed_data/DF_CENTROID.csv')
df_centroid['fips'] = df_centroid['fips'].astype(str).str.zfill(3)

pairs = pd.DataFrame(
    list(product(df_centroid.fips, df_centroid.fips)
), columns=["fips_origin", "fips_dest"])

# Merge coordinates for county A
pairs = pairs.merge(df_centroid[['fips','centroid_x','centroid_y']], left_on='fips_origin', right_on='fips')
pairs.rename(columns={'centroid_x':'x_origin','centroid_y':'y_origin'}, inplace=True)
pairs.drop(columns='fips', inplace=True)

# Merge coordinates for county B
pairs = pairs.merge(df_centroid[['fips','centroid_x','centroid_y']], left_on='fips_dest', right_on='fips')
pairs.rename(columns={'centroid_x':'x_dest','centroid_y':'y_dest'}, inplace=True)
pairs.drop(columns='fips', inplace=True)

# Euclidean distance in meters
pairs['distance_m'] = np.sqrt((pairs['x_dest'] - pairs['x_origin'])**2 + (pairs['y_dest'] - pairs['y_origin'])**2)
pairs['distance_mi'] = pairs['distance_m'] / 1609.34
pairs['d_ij'] = pairs['distance_mi']

pairs['county_i_name'] = pairs['fips_origin'].map(d_fips_county)
pairs['county_j_name'] = pairs['fips_dest'].map(d_fips_county)

# pairs stores pair-wise distances between county centroids in miles
pairs = pairs[['county_i_name', 'county_j_name', 'd_ij']]
# pairs.head()

In [4]:
# Using web-widget here, we can estimate the diagonals (within county commute LA-LA commute)
# https://www.census.gov/acs/www/about/why-we-ask-each-question/commuting/
# We can then multiply by some scalar factor
# LA: 30 min commute
# OC: 27 min commute
# San Bernardino: 33 minute commute
# Ventura: 26 min commute
# Riverside: 34 min commute
# San Diego: 26 min commute

# in minutes (median from ACS census widget)
factor = 0.25
d_inter_county_commute = {
    'Los Angeles': 30,
    'Orange': 27,
    'San Bernardino': 33,
    'San Diego': 26,
    'Riverside': 34,
    'Ventua': 26
}

# min per mile (factors suggested by chatgpt)
d_inter_county_commute_factor = {
    'Los Angeles': 2.3,
    'Orange': 2.0,
    'San Bernardino': 1.6,
    'San Diego': 2.0,
    'Riverside': 1.6,
    'Ventua': 1.7
}

for k, v in d_inter_county_commute.items():
    factor = d_inter_county_commute_factor[k]
    pairs.loc[
        (pairs['county_i_name']==k) & 
        (pairs['county_j_name']==k)
    , 'd_ij'] = v / factor

In [5]:
# Merge commuting distance matrix d_ij into dataframe
df_model_ij = pd.merge(
    df_model_ij, pairs, how='left', on=['county_i_name','county_j_name'])

df_model_ij.head()

,county_i,county_j,county_i_name,county_j_name,L_ij_data,d_ij
0,0,0,Los Angeles,Los Angeles,4429.523,13.043478
1,1,0,Orange,Los Angeles,180.250,32.571940
2,2,0,Riverside,Los Angeles,53.172,74.016584
3,3,0,San Bernardino,Los Angeles,132.992,54.615943
4,4,0,San Diego,Los Angeles,6.075,105.441086


### Add.l Parameters: $F_i$

F: In estimating wildfire exposure of LA, we think about the approximate proportion of structures that were destroyted by the wildfires. If we make the assumption that 10,000 of 1 million structures were destroyed by the fires:
\begin{align}
&F_i = 0.01, \hspace{0.3em} i = \mathrm{Los Angeles}\\
&F_i = 0.00 \hspace{1em} \mathrm{otherwise} \\
\end{align}

In [6]:
d_Fi_nofire = {
    'Los Angeles':0, 
    'Orange':0, 
    'Riverside':0, 
    'San Bernardino':0,
    'San Diego':0, 
    'Ventura':0
}
d_Fi_fire1 = {
    'Los Angeles':0.01, 
    'Orange':0, 
    'Riverside':0, 
    'San Bernardino':0,
    'San Diego':0, 
    'Ventura':0
}
df_model_i['F_i_nofire'] = df_model_i['county_i_name'].map(d_Fi_nofire)
df_model_i['F_i_fire1'] = df_model_i['county_i_name'].map(d_Fi_fire1)

In [7]:
kappa = 0.08 # Redding 2016: 0.05–0.15 per mile, 3-10% wage loss per 10 miles
chi = 0.5 # Calibrate this: low chi: small fire response, high fire chi: large response
eta = 0.7 # Housing supply elasticity saiz 2010

We can come back to estimate $\kappa$ from the gravity eqns, but for now we will set it to 0.08 and move on
\begin{align}
L_{ij}^{data} = \pi_{ij} N
\end{align}

We have now built $d_{ij}$, $F_i$, $Z_j$ from wages.   
We will solve for baseline equilibrium.   
Then increase $F_{LA}$, and re-solve for equilibrium.   
Compare population, wages, housing, and commuting.


### Back out baseline housing prices

At spatial equilibrium:  
- labor markets clear
- housing markets clear
- commuting flows reflect choices
  households are indifferent across residence-workplace pairs

Housing market clearing:
\begin{align}
p_i = \left(\frac{N_i}{H_i}\right)^{1/\eta}
\end{align}

In [8]:
def compute_prices(N_i, H_i, eta):
    p_i = (N_i / H_i) ** (1/eta)
    return p_i
    
df_model_i['p_i_eq'] = compute_prices(
    df_model_i['N_i_2024_data'], 
    df_model_i['H_i_2024_data'], 
    eta
)

In [9]:
# df_model_i[['county_i_name', 'p_i_eq']].sort_values(by='p_i_eq', ascending=False)

We notice that $p_i$ is not super aligned with median county home prices rankings.
We note that we are unsure whether reported population that is used to estimate number of households include unhoused individuals/families or not.

### Normalize wages w_j for Z_j

In [10]:
def compute_wages_norm(L_j, w_j_emp):
    weights = L_j / L_j.sum()
    w_bar = np.sum(weights * w_j_emp) # weighted avg
    w_j_emp_norm = w_j_emp / w_bar
    return w_j_emp_norm

# normalize weighted avg wage per employee
weights = df_model_j['L_j_data'] / df_model_j['L_j_data'].sum()
w_bar = np.sum(weights * df_model_j['w_j_emp_data']) # weighted avg
df_model_j['w_j_emp_data'] / w_bar

df_model_j['w_j_emp_norm_data'] = compute_wages_norm(
    df_model_j['L_j_data'], df_model_j['w_j_emp_data']
)

w_per_capita = df_model_j['w_j_county_data'] / df_model_j['L_j_data']
w_bar_capita = np.sum(weights * w_per_capita)
df_model_j['w_j_county_norm_data'] = w_per_capita / w_bar_capita

### Back out Amenities
By definition we have a relation between $L_i$, working residents living in county i, and $\pi_{ij}$, the choice probabilities of residents living in county i and working in county j. We can treat both the denominator sum on the r.h.s. and N on the l.h.s. as a fixed constant C, and solve for amenities $A_i$ up to that fixed constant.

\begin{align}
L_i &= L \displaystyle \sum_{j} \pi_{ij}\\
\cfrac{L_i}{L} &= \displaystyle \sum_{j} \cfrac{\exp ({\log w_j - \kappa d_{ij} - \log p_i + A_i - \chi F_i) }}{ \sum_k \exp ({\log w_k - \kappa d_{ik} - \log p_i + A_i - \chi F_i) }}\\
A_i &= \log {L_i}  - \log \left( \sum_j \exp(\log w_j - \kappa d_{ij} - \log P_i - \chi F_i) \right) + C
\end{align}

When we are solving for baseline, the $F_i$ vector is 1.

Something to be careful of: have to be consistent with how we think about $N_{i}$ and $L_{i} = \sum_j L_{ij}$. We re-structured N to be number of households (given avg household size in each county in `cleaning_rf.ipynb`), and L is number of workers in a county. This gives us a rough estimate on workers per household.

In [11]:
df_model_i[['county_i', 'N_i_2024_data', 'L_i_data']].sort_values(
    by='N_i_2024_data', ascending=False) 

_ = df_model_i.copy()
# _['N_i_2024_mils'], _['L_i_mils'] = _['N_i_2024_data'], _['L_i_data']
_['workers_per_household'] = _['L_i_data']/_['N_i_2024_data']
_[['county_i_name', 'N_i_2024_data', 'L_i_data', 'workers_per_household']]

,county_i_name,N_i_2024_data,L_i_data,workers_per_household
0,Los Angeles,3304.913758,4742.279,1.434918
1,Orange,1056.690000,1547.549,1.464525
2,Riverside,835.918456,1016.166,1.215628
3,San Bernardino,698.524127,909.384,1.301865
4,San Diego,1214.418315,1615.443,1.330220
5,Ventura,272.428618,389.548,1.429909


We observe that $L_i$ is slightly larger than $N_i$ meaning that slightly more than one person works in each household.

In [12]:
# merge ij, i, j dataframes for \pi_ij computation
df_model_merge = df_model_ij.copy()
df_model_merge = df_model_merge.merge(
    df_model_i, on=['county_i_name','county_i'], how='left'
    ).merge(df_model_j,on=['county_j_name','county_j'])

# # data commuting shares
# df_model_merge['pi_ij_data'] = (
#     df_model_merge['L_ij_data'] /
#     df_model_merge.groupby('county_i')['L_ij_data'].transform('sum')
# )
# # invert A_i from shares
# _df = df_model_merge.copy()
# _df['_exp'] = np.exp(
#     np.log(_df['w_j_emp_norm_data'])
#     - kappa * _df['d_ij']
#     - np.log(_df['p_i_eq'])
#     - chi * _df['F_i_nofire']
# )
# df_model_i['A_i_eq'] = (
#     np.log(
#         _df.groupby('county_i')['pi_ij_data'].sum()
#     )
#     - np.log(
#         _df.groupby('county_i')['_exp'].sum()
#     )
# )
# if 'A_i_eq' not in df_model_merge.columns:
#     df_model_merge = df_model_merge.merge(df_model_i[['county_i', 'A_i_eq']], how='left', on='county_i')
# ---------

# constructing first term (vector in i)
_df_i = df_model_i.copy()
_df_i['_term1'] = np.log(
    df_model_ij.groupby('county_i')['L_ij_data'].sum()
)

# constructing second term 
_df_ij = df_model_merge.copy()
_df_ij['_exp'] = np.exp(
    np.log(df_model_merge['w_j_emp_data']) - \
    kappa * df_model_merge['d_ij'] - \
    np.log(df_model_merge['p_i_eq']) - \
    chi * df_model_merge['F_i_nofire']
)
_df_i['_term2'] = np.log(_df_ij.groupby('county_i')['_exp'].sum())
_df_i['A_i_eq'] = _df_i['_term1'] - _df_i['_term2']
_df_i['A_i_eq'] = _df_i['A_i_eq'] - _df_i['A_i_eq'].mean() # normalize 

In [13]:
if 'A_i_eq' not in df_model_i.columns:
    df_model_i = df_model_i.merge(_df_i[['county_i', 'A_i_eq']], how='left', on='county_i')

if 'A_i_eq' not in df_model_merge.columns:
    df_model_merge = df_model_merge.merge(df_model_i[['county_i', 'A_i_eq']], how='left', on='county_i')
    
df_model_i[['county_i_name', 'A_i_eq']
    ].sort_values(by='A_i_eq', ascending=False)

,county_i_name,A_i_eq
0,Los Angeles,1.089167
2,Riverside,0.396876
4,San Diego,0.354712
3,San Bernardino,0.155105
1,Orange,0.027650
5,Ventura,-2.023510


### Compute baseline $\pi_{ij}$

- We could either do joint probability $\pi_{i, j}$
- or do conditional probability $\pi_{j|i}$
- We model residence–workplace choice as a multinomial logit over all (i,j) pairs. This formulation is equivalent to a two-step model where households choose residence i and then workplace j conditional on residence, as in Rossi-Hansberg (2019).


In [14]:
def stable_softmax(x):
    x_max = np.max(x)
    e_x = np.exp(x - x_max)
    return e_x / e_x.sum()

def compute_U_ij(w_j, p_i, d_ij, A_i, F_i, kappa, chi):
    U_ij = np.log(w_j)  - np.log(p_i) \
        - kappa * d_ij + A_i - chi * F_i
    return U_ij
    
df_model_merge['U_ij_eq'] = compute_U_ij(
    df_model_merge['w_j_emp_norm_data'],
    df_model_merge['p_i_eq'],
    df_model_merge['d_ij'],
    df_model_merge['A_i_eq'],
    df_model_merge['F_i_nofire'],
    kappa, chi
)


# ---- joint probability over all (i,j) ----
df_model_merge['pi_ij_joint_eq'] = stable_softmax(df_model_merge['U_ij_eq'])
# scipy.special.softmax(df_model_merge['U_ij_eq'])

# ---- conditional probability, given residence i ----
# softmax over j within each county_i
df_model_merge['pi_ij_cond_eq'] = \
    df_model_merge.groupby('county_i')['U_ij_eq'].transform(
        # lambda x: scipy.special.softmax(x.values)
        lambda x: stable_softmax(x.values)
        
)
# ---- predicted equilibrium Lij given backed out pi_ij ----
# This is what we use to compare to our counterfactual!!!
df_model_merge['L_ij_eq'] = df_model_merge['pi_ij_cond_eq'] * df_model_merge['L_i_data']

# merge pi's and U back to
df_model_ij = df_model_ij.merge(df_model_merge[
    ['county_i', 'county_j', 'U_ij_eq','pi_ij_cond_eq', 'pi_ij_joint_eq', 'L_ij_eq']
], on=['county_i', 'county_j'])

In [15]:
# # ------------------- Assess predicted Lij and Lij data
_ = df_model_merge.copy()

# # _['predicted_L_ij_eq'] = _['pi_ij_cond_eq'] * _['N_i_2024_data']
# _['predicted_L_ij_eq'] = _['pi_ij_cond_eq'] * _['L_i_data']

# _['%_diff_pi_L'] = (_['predicted_L_ij_eq'] - _['L_ij_data']) / _['L_ij_data'] * 100
# # _[['county_i_name', 'county_j_name', 'L_ij_data', 'predicted_L_ij_eq', 'pi_ij_joint_eq', '%_diff_pi_L']]

For most commuting flows, our observed $L_{ij}^{\mathrm{model}} ~= L_{ij}^{\mathrm{data}}$. 50% of our predicted commuting flows are within -55% to 131% off. Let's just move on, and keep in mind that we should always compare counterfactuals to the model's baseline predictions for $L_{\mathrm{ij, predicted baseline}}$ and $L_{\mathrm{ij, counterfactual}}$, not$L_{\mathrm{ij, data}}$ and $L_{\mathrm{ij, counterfactual}}$
Same for $N_i$, 

Once we are in structural / spatial equilibrium mode, many objects become model-implied, not data objects anymore.

In [16]:
scatter = alt.Chart(_).mark_circle(color='#CD5C5C').encode(
    x=alt.X('L_ij_data', 
            # scale=alt.Scale(type='symlog'), 
            title='Observed L_ij (in thousands)'),
    y=alt.Y('L_ij_eq', 
            # scale=alt.Scale(type='symlog'),
            title='Predicted L_ij (in thousands)'),
    tooltip=['county_i_name', 'county_j_name', 'L_ij_data', 'L_ij_eq']
).properties(
    width=400,
    height=350,
    title='Observed vs Predicted Commuting Flows'
)
# y = x line
max_val = max(_['L_ij_data'].max(), _['L_ij_eq'].max())
line = alt.Chart(pd.DataFrame({'x':[0, max_val], 'y':[0, max_val]})).mark_line(
    color='#898989', strokeDash=[5,5], strokeWidth=.5
).encode(
    x='x',
    y='y'
)
chart_obs_pred = line + scatter
# chart_obs_pred.save('output/chart_obs_pred_lin.svg')
chart_obs_pred.interactive()

alt.LayerChart(...)

This chart is mainly a sanity check that our baseline parameters from model inversion are roughly estimating $\pi_{ij} \hspace{0.3em} \mathrm{s.t.} \hspace{0.3em} L_{ij}^{\mathrm{model}} = L_{ij}^{\mathrm{data}}$

### Running the counterfactual

- Baseline equilibrium: ${N_i^{(0)}, L_{ij}^{(0)}, w_j^{(0)}, p_i^{(0)}, \pi_{ij}^{(0)}, }$
- Change $Fi$
- Recompute utilities $U_{ij}^{(1)} =  \log {w_j^{(1)}} - \log {p_i^{(1)}} \kappa d_{ij} + A_i - \chi F_i$ 
- Update choices $\pi_{ij}^{(1)} = \mathrm{softmax_j} (U_{ij}^{(1)})$
- Update implied worker locations $L_i^{(1)} =  L \sum_j \pi_{ij}^{(1)}$
- Update implied populations $N_i^{(1)} = L_i^{(1)}/\lambda_i$
- Update housing prices $p_i^{(1)} = \left( \cfrac{N_i^{(1)}}{H_i^{(1)}} \right)^ \eta$
- Labor supply changes wages: $w_j^{(1)} = Z_j (L_j^{(1)})^ {-\alpha}$

Iterate ^ until $N_i^{(t+1)} - N_i^{(t)}$ converges

In [17]:
def compute_prices(N_i, H_i, eta):
    p_i = (N_i / H_i) ** (1/eta)
    return p_i

def compute_wages_norm(L_j, w_j_emp):
    weights = L_j / L_j.sum()
    w_bar = np.sum(weights * w_j_emp) # weighted avg
    w_j_emp_norm = w_j_emp / w_bar
    return w_j_emp_norm


In [18]:
df_i_cf = df_model_i.copy()          # county-level
df_j_cf = df_model_j.copy()          # county-level
df_ij_cf = df_model_merge.copy()     # commuting matrix

df_i_cf = df_i_cf.rename(columns={
    'N_i_2024_data':'N_i', # initialize values (but they will change in loop)
    'L_i_data':'L_i',      # initialize values (but they will change in loop)
    'p_i_eq':'p_i',        # initialize values (but they will change in loop)
    'A_i_eq':'A_i',
    'H_i_2024_data':'H_i',
})[['county_i_name', 'county_i', 'H_i', 'N_i', 'L_i', 'F_i_fire1', 'p_i', 'A_i']]

df_j_cf = df_j_cf.rename(columns={
    'w_j_emp_data':'w_j_emp', 
    'L_j_data':'L_j',
    'w_j_emp_norm_data':'w_j_emp_norm',
})[['county_j_name', 'county_j', 'w_j_emp','w_j_emp_norm', 'L_j']]

df_ij_cf = df_ij_cf.rename(columns={
    'L_ij_data':'L_ij',
    'U_ij_eq':'U_ij',
    'pi_ij_joint_eq':'pi_ij_joint',
    'pi_ij_cond_eq':'pi_ij_cond',
})[['county_i', 'county_j', 'county_i_name', 'county_j_name', 
    'L_ij', 'd_ij', 'U_ij', 'pi_ij_joint', 'pi_ij_cond']]

# re-merge all 'i' vars (in particular housing has changed)
# this way any counterfactual changes can just be df_i and df_ij will re-merge i-level data
df_ij_cf = df_ij_cf.merge(df_i_cf, how='left', on=['county_i', 'county_i_name'])
df_ij_cf = df_ij_cf.merge(df_j_cf, how='left', on=['county_j', 'county_j_name'])

In [19]:
def solve_equilibrium(
    df_i, df_j, df_ij,
    kappa, 
    chi, # fire coefficient
    structure_shock=0.01,
    max_iter=500, 
    tol=1e-6
):
    df_i = df_i.copy()
    df_j = df_j.copy()
    df_ij = df_ij.copy()

    # Setting housing stock to 0.99 of original
    H_i_eq_LA = df_i.loc[df_i['county_i_name']=='Los Angeles', 'H_i'].values[0]
    df_i.loc[df_i['county_i_name']=='Los Angeles', 'H_i'] = H_i_eq_LA * (1-structure_shock)


    for it in range(max_iter):
        # ---- compute utilities ----
        df_ij['U_ij'] = compute_U_ij(
            df_ij['w_j_emp_norm'],
            df_ij['p_i'],
            df_ij['d_ij'],
            df_ij['A_i'],
            df_ij['F_i_fire1'],
            kappa, chi
        )

        # ---- conditional choice ----
        df_ij['pi_ij_cond'] = df_ij.groupby('county_i')['U_ij'].transform(
            lambda x: stable_softmax(x.values)
        )

        # ---- implied commuting flows ---- # <-- Li or Ni??
        df_ij['L_ij'] = df_ij['pi_ij_cond'] * df_ij['N_i']

        # ---- update employment by workplace ----
        L_j = df_ij.groupby('county_j')['L_ij'].sum().rename('L_j')

        # ---- update prices (housing market clearing) ----
        df_j = df_j.drop(columns=['L_j']) # drop previous values
        df_j = df_j.merge(L_j, left_on='county_j', right_index=True, how='left')
        p_i_new = (df_i['N_i'] / df_i['H_i']) ** eta

        # ---- convergence check ----
        err = np.max(np.abs(p_i_new - df_i['p_i']))
        df_i['p_i'] = 0.7 * df_i['p_i'] + 0.3 * p_i_new  # damping

        if err < tol:
            print(f"Converged in {it} iterations")
            break

    return df_i, df_j, df_ij

The above answers the question: Given a housing shock in LA, how do commuting patterns and prices adjust holding population and wages fixed? This is a short-run equilibrium in which population is fixed and commuting patterns and housing prices adjust endogenously. 



In [20]:
df_i_cf, df_j_cf, df_ij_cf = solve_equilibrium(df_i_cf, df_j_cf, df_ij_cf, kappa, chi=2)

Converged in 32 iterations


In [21]:
# only p_i changes in this df
df_compare_i = df_model_i.merge(
    df_i_cf[['county_i_name','county_i', 'N_i', 'L_i', 'p_i']].rename(
        columns={'p_i':'p_i_cf'})[['county_i_name', 'p_i_cf']], how='left', on='county_i_name'
)
df_compare_i['diff_p_i'] = df_compare_i['p_i_cf'] - df_compare_i['p_i_eq']

# only L_j changes in this dataframe
df_compare_j = df_model_j.merge(
    df_j_cf.rename(columns={'L_j':'L_j_cf'}
    )[['county_j_name', 'L_j_cf']], how='left', on='county_j_name'
)
df_compare_j['diff_L_j'] = df_compare_j['L_j_cf'] - df_compare_j['L_j_data']

In [22]:
# Uij changes only for LA-X
# L_ij changes significantly
# pi_ij change very minutely - chat says microscopic chagnes in pi_ij can have huge changes in L_ij
df_compare_ij =df_model_ij.merge(df_ij_cf[['county_i', 'county_j', 'L_ij', 'U_ij','pi_ij_joint', 'pi_ij_cond']].rename(columns={
    'L_ij':'L_ij_cf',
    'U_ij':'U_ij_cf',
    'pi_ij_joint':'pi_ij_joint_cf',
    'pi_ij_cond':'pi_ij_cond_cf'
}), how='left', on=['county_i', 'county_j'])

df_compare_ij = df_compare_ij[[
    'county_i_name', 'county_j_name', 'L_ij_eq','L_ij_cf', 
   'U_ij_eq', 'U_ij_cf', 'pi_ij_cond_eq', 'pi_ij_cond_cf', 
   'pi_ij_joint_eq', 'pi_ij_joint_cf'
]]

df_compare_ij['diff_U_ij'] = df_compare_ij['U_ij_cf'] - df_compare_ij['U_ij_eq']
df_compare_ij['diff_pi_ij_joint'] = df_compare_ij['pi_ij_joint_cf'] - df_compare_ij['pi_ij_joint_eq']
df_compare_ij['diff_pi_ij_cond'] = df_compare_ij['pi_ij_cond_cf'] - df_compare_ij['pi_ij_cond_eq']
df_compare_ij['diff_L_ij'] = df_compare_ij['L_ij_cf'] - df_compare_ij['L_ij_eq']

# df_compare_ij[['county_i_name','county_j_name', *[x for x in df_compare_ij.columns if 'diff' in x]]]

In [23]:
L_i = df_compare_ij.groupby('county_i_name')['L_ij_eq'].sum().reset_index(name='L_i_eq')
L_j = df_compare_ij.groupby('county_j_name')['L_ij_eq'].sum().reset_index(name='L_j_eq')

df_compare_i = df_compare_i.merge(L_i, how='left', on='county_i_name')
df_compare_j = df_compare_j.merge(L_j, how='left', on='county_j_name')

In [27]:
df_compare_ij.to_csv('processed_data/DF_COMPARE_IJ.csv', index=False)
df_compare_i.to_csv('processed_data/DF_COMPARE_I.csv', index=False)
df_compare_j.to_csv('processed_data/DF_COMPARE_J.csv', index=False)

In [ ]:
# plot changes in p_i, plot L_j, and commuting L_ij differences